In [3]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

In [4]:
hf_df = pd.read_excel("Data/hedge_funds_returns_data.xlsx")

short_mapping = {
    'Date': 'Date',
    'HFRI 400 (US) Fund Weighted Composite Index (HFRI4FWC)': 'HFRI4FWC',
    'HFRI 400 (US) EH: Long/Short Index (HFRI4ELS)': 'HFRI4ELS',
    'HFRI 400 (US) EH: Fundamental Value Index (HFRI4EHV)': 'HFRI4EHV',
    'HFRI 400 (US) Event-Driven Index (HFRI4ED)': 'HFRI4ED'
    
}

hf_df = hf_df.rename(columns=short_mapping)
hf_df.head()

,Date,HFRI4FWC,HFRI4ELS,HFRI4ED,HFRI4EHV
0,2005-01-31,0.0047,0.0067,0.0012,0.0049
1,2005-02-28,0.0198,0.0279,0.0111,0.0219
2,2005-03-31,-0.0103,-0.0172,0.0002,-0.0111
3,2005-04-29,-0.0124,-0.0165,-0.0105,-0.0137
4,2005-05-31,0.0079,0.0122,0.0064,0.0122


In [5]:
def clean_data(df):
    df = df.set_index('Date')
    df = df.dropna()
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df
    
hf_df = clean_data(hf_df)

In [6]:
HFRI4FWC_beta = pd.read_excel("all_output_results/All_Hedge_Fund_Betas/HFRI4FWC_betas.xlsx")
HFRI4ELS_beta = pd.read_excel("all_output_results/All_Hedge_Fund_Betas/HFRI4ELS_betas.xlsx")
HFRI4ED_beta = pd.read_excel("all_output_results/All_Hedge_Fund_Betas/HFRI4ED_betas.xlsx")
HFRI4EHV_beta = pd.read_excel("all_output_results/All_Hedge_Fund_Betas/HFRI4EHV_betas.xlsx")

In [3]:
def clean_data(df):
    df = df.set_index('Quarter')
    df = df.dropna()
    df.index = pd.to_datetime(df.index.str.replace(' Q', 'Q')) 
    df = df.sort_index()
    return df

In [ ]:
HFRI4FWC_beta = clean_data(HFRI4FWC_beta)
HFRI4ELS_beta = clean_data(HFRI4ELS_beta)
HFRI4ED_beta = clean_data(HFRI4ED_beta)
HFRI4EHV_beta = clean_data(HFRI4EHV_beta)

/var/folders/ky/n1n3tvzn3zb6_44x8nfzwn700000gn/T/ipykernel_71415/3619432106.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index.str.replace(' Q', 'Q'))
/var/folders/ky/n1n3tvzn3zb6_44x8nfzwn700000gn/T/ipykernel_71415/3619432106.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index.str.replace(' Q', 'Q'))
/var/folders/ky/n1n3tvzn3zb6_44x8nfzwn700000gn/T/ipykernel_71415/3619432106.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index.str.replace(' Q', 'Q'))
/var/folders/ky/n1n

In [5]:
def get_beta_ranges(df, buffer=0.05):
    ranges = {}
    
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        min_val = df[col].min()
        max_val = df[col].max()
        q10, q25, q75, q90 = df[col].quantile([0.1, 0.25, 0.75, 0.9])
        
        ranges[col] = {
            'conservative': [q25, q75],
            'moderate': [q10, q90],
            'aggressive': [mean - 1.5*std, mean + 1.5*std],
            'historical': [min_val + buffer, max_val - buffer]
        }
    
    return ranges

In [6]:
get_beta_ranges(HFRI4FWC_beta)

{'β_Mkt-RF': {'conservative': [0.1729256245634997, 0.4008870053713103],
  'moderate': [0.08777342016761672, 0.5477569142070862],
  'aggressive': [-0.1587392029229776, 0.8102648950215637],
  'historical': [-0.2405588629492254, 1.653534777651084]},
 'β_SMB': {'conservative': [-0.232254808626075, 0.44672131147541],
  'moderate': [-0.5071239717390156, 0.7384212538138806],
  'aggressive': [-2.2984688518250715, 2.1907557483562554],
  'historical': [-8.254347826086956, 3.327758274824474]},
 'β_HML': {'conservative': [-0.1529573917778382, 0.3738078639653301],
  'moderate': [-0.5037642022315915, 0.7207601546313963],
  'aggressive': [-0.9876969697866784, 1.3192665884760106],
  'historical': [-1.49296087131908, 3.6277251184834114]},
 'β_Mom': {'conservative': [-0.2719524634268228, 0.155590886662855],
  'moderate': [-0.4651156578552757, 0.3122749053302649],
  'aggressive': [-0.6655497035539654, 0.5775472831682275],
  'historical': [-0.8940463065049615, 1.608056042031524]}}

In [7]:
ranges = get_beta_ranges(HFRI4FWC_beta)

# Access specific ranges
print(ranges['β_Mkt-RF']['conservative'])  # [0.35, 0.45]
print(ranges['β_SMB']['moderate'])         # [0.2, 0.85]

[0.1729256245634997, 0.4008870053713103]
[-0.5071239717390156, 0.7384212538138806]


In [18]:
def find_optimal_ranges_multi_assets(beta_dfs_dict, returns_df, risk_free_rate=0.02):
    """
    Find optimal beta ranges using multiple assets' betas and their returns
    
    Parameters:
    beta_dfs_dict: Dictionary of DataFrames {'asset_name': beta_df, ...}
    returns_df: DataFrame with returns for all assets (columns should match asset names)
    risk_free_rate: Annual risk-free rate
    """
    
    # Get ranges from first asset (assuming similar factor structure)
    first_asset = list(beta_dfs_dict.keys())[0]
    ranges = get_beta_ranges(beta_dfs_dict[first_asset])
    
    results = {}
    
    # For each factor, test ranges across all assets
    for factor in beta_dfs_dict[first_asset].columns:
        factor_results = {}
        
        for range_type in ['conservative', 'moderate', 'aggressive', 'historical']:
            lower, upper = ranges[factor][range_type]
            
            all_returns_in_range = []
            total_periods = 0
            
            # Check each asset
            for asset_name, beta_df in beta_dfs_dict.items():
                if factor in beta_df.columns and asset_name in returns_df.columns:
                    
                    # Convert to period index for matching
                    beta_df_copy = beta_df.copy()
                    beta_df_copy.index = pd.PeriodIndex(beta_df_copy.index, freq='Q')
                    
                    returns_df_copy = returns_df.copy()
                    returns_df_copy.index = pd.PeriodIndex(returns_df_copy.index, freq='M')
                    
                    # Find quarters where this asset's beta is in range
                    in_range_quarters = beta_df_copy[(beta_df_copy[factor] >= lower) & 
                                                   (beta_df_copy[factor] <= upper)].index
                    
                    if len(in_range_quarters) > 0:
                        # Get monthly returns for this asset in those quarters
                        for quarter in in_range_quarters:
                            quarter_months = returns_df_copy[
                                returns_df_copy.index.to_timestamp().to_period('Q') == quarter
                            ][asset_name]
                            
                            if not quarter_months.empty:
                                all_returns_in_range.extend(quarter_months.values)
                        
                        total_periods += len(in_range_quarters)
            
            # Calculate combined performance metrics
            if len(all_returns_in_range) > 0:
                combined_returns = pd.Series(all_returns_in_range)
                
                avg_monthly_return = combined_returns.mean()
                monthly_risk = combined_returns.std()
                
                # Annualized metrics
                annual_return = (1 + avg_monthly_return) ** 12 - 1
                annual_risk = monthly_risk * np.sqrt(12)
                sharpe = (annual_return - risk_free_rate) / annual_risk if annual_risk > 0 else 0
                
                factor_results[range_type] = {
                    'range': [lower, upper],
                    'total_observations': len(all_returns_in_range),
                    'total_quarters': total_periods,
                    'avg_monthly_return': avg_monthly_return,
                    'annual_return': annual_return,
                    'annual_risk': annual_risk,
                    'sharpe': sharpe
                }
        
        results[factor] = factor_results
    
    return results

# Usage:
beta_assets = {
    'HFRI4FWC': HFRI4FWC_beta,
    'HFRI4ELS': HFRI4ELS_beta, 
    'HFRI4ED': HFRI4ED_beta,
    'HFRI4EHV': HFRI4EHV_beta
}

optimal_results = find_optimal_ranges_multi_assets(beta_assets, hf_df)

In [19]:
# Find best ranges across all assets
best_ranges = {}
for factor in optimal_results:
    best_sharpe = -999
    best_range_type = None
    
    for range_type in optimal_results[factor]:
        if optimal_results[factor][range_type]['sharpe'] > best_sharpe:
            best_sharpe = optimal_results[factor][range_type]['sharpe']
            best_range_type = range_type
    
    if best_range_type:
        best_ranges[factor] = {
            'best_type': best_range_type,
            'range': optimal_results[factor][best_range_type]['range'],
            'sharpe': best_sharpe,
            'observations': optimal_results[factor][best_range_type]['total_observations']
        }

print("Optimal Beta Ranges (All Assets Combined):")
for factor, info in best_ranges.items():
    lower, upper = info['range']
    print(f"{factor}: [{lower:.3f}, {upper:.3f}] (Sharpe: {info['sharpe']:.3f}, Obs: {info['observations']})")

Optimal Beta Ranges (All Assets Combined):
β_Mkt-RF: [0.173, 0.401] (Sharpe: 0.816, Obs: 198)
β_SMB: [-0.232, 0.447] (Sharpe: 1.460, Obs: 246)
β_HML: [-0.153, 0.374] (Sharpe: 1.223, Obs: 237)
β_Mom: [-0.272, 0.156] (Sharpe: 0.936, Obs: 222)
